In [146]:
import openpyxl

key_file = openpyxl.open("auth/weather_key.xlsx")
key_dict = {}

excel_sheet = key_file.active
for key in excel_sheet.rows:
    key_dict[key[0].value] = key[1].value

key_file.close()

In [147]:
from urllib.parse import urlencode
import requests as req
import pprint as pp
import pandas as pd
import re
from datetime import datetime as dt

init_page_no = 1
query_dict = dict({
    "serviceKey" : key_dict['decoding-key'],
    "numOfRows" : 10,
    "pageNo": init_page_no,
    "dataCd": "ASOS",
    "dateCd": "DAY",
    "startDt": "20170101",
    "endDt": "20181231",
    "stnIds": "108",
    "dataType": "JSON"
})    

api_url = "http://apis.data.go.kr/1360000/AsosDalyInfoService/getWthrDataList?"
datas = []
while True:
    if (len(datas) % 50) == 0:
        print("Page: {} / Count: {}".format(query_dict['pageNo'], len(datas)))
    query = urlencode(query_dict)
    res = req.get(api_url + query)
    body = res.json()['response']['body']
    
    items = body['items']['item']
    page_info = dict({
        "num_of_rows" : body['numOfRows'],
        "page_no" : body['pageNo'],
        "total_count" : body['totalCount']
    })
    for item in items:
        in_data = dict({
            "날짜": dt.strptime(item['tm'], '%Y-%m-%d'),
            "날씨": item['iscs'],
            "평균온도": item['avgTa'],
            "평균습도": item['avgRhm']
        })
        datas.append(in_data)
    
    if len(datas) >= page_info['total_count']:
        break
    else:
        query_dict['pageNo'] = (body['pageNo'] + 1)

Page: 1 / Count: 0
Page: 6 / Count: 50
Page: 11 / Count: 100
Page: 16 / Count: 150
Page: 21 / Count: 200
Page: 26 / Count: 250
Page: 31 / Count: 300
Page: 36 / Count: 350
Page: 41 / Count: 400
Page: 46 / Count: 450
Page: 51 / Count: 500
Page: 56 / Count: 550
Page: 61 / Count: 600
Page: 66 / Count: 650
Page: 71 / Count: 700


In [148]:
# 눈 만 온 경우,
# 비 만 온 경우,
# ** 눈이랑 비가 같이 온 경우 (진눈깨비 관건,,)
weather_dict = dict()
for data in datas:
    data_weather = data['날씨']
    if "비" in data_weather or "눈" in data_weather:
        data_weather = data_weather.replace("안개비", "비")
        data_weather = data_weather.replace("진눈깨비", "눈")
        data_weather = data_weather.replace("소나기", "비")
        data_weather = data_weather.replace("싸락눈", "눈")
        
        date = data["날짜"]
        weather_list = []
        print(date)
        rain_idx = data_weather.find("비")
        snow_idx = data_weather.find("눈")
        weather_idx = 0
                    
        if rain_idx != -1:
            rain_idx -= 1
        if snow_idx != -1:
            snow_idx -= 1
        
        if rain_idx == -1:
            weather_idx = snow_idx
        elif snow_idx == -1:
            weather_idx = rain_idx
        elif (rain_idx != -1) and (snow_idx != -1):
            weather_idx = snow_idx if rain_idx > snow_idx else rain_idx
            
        weathers = data_weather[weather_idx:]
        for weather in weathers.split("."):
            w = weather.split(" ")
            for _ in w:
                if _ == "" or "박무" in _ or "연무" in _ or "황사" in _:
                    continue
                else:
                    weather_list.append(_.strip())
        
        weather_dict[date] = weather_list
        print(weather_dict[date])
        print()

2017-01-02 00:00:00
['{비}0410-0550', '{비}0725-0750']

2017-01-08 00:00:00
['{비}1515-1545', '{비}1925-1950', '{비}2130-2250']

2017-01-09 00:00:00
['{비}0205-0220', '{비}0350-0445']

2017-01-12 00:00:00
['{눈}0040-0220']

2017-01-13 00:00:00
['{눈}0820-{눈}{강도0}0900-1145']

2017-01-18 00:00:00
['{눈}1225-1315']

2017-01-20 00:00:00
['{눈}0210-{눈}{강도0}0300-{눈}{강도1}0600-0855']

2017-01-21 00:00:00
['{눈}0845-{눈}{강도0}0900-0915', '{눈}1125-1158', '{눈}1415-{눈}{강도0}1500-{시정(미만)}{1km}{눈}1715-{시정(이상)}{1km}{눈}1724-{눈}{강도0}1800-1845', '{눈}1945-1958', '{눈}2303-{시정(미만)}{1km}{눈}2333-{시정(미만)}{0', '5km}{눈}2336-{시정(이상)}{0', '5km}{눈}2345-{시정(이상)}{1km}{눈}2351-{눈}{강도0}2400-']

2017-01-22 00:00:00
['{눈}-0025']

2017-01-26 00:00:00
['{비}2310-{비}{강도0}2400-']

2017-01-27 00:00:00
['{비}-0140']

2017-01-29 00:00:00
['{눈}0405-0417', '{눈}1015-{눈}1110-{눈}1150-{눈}{강도0}1200-{눈}1240-{눈}1440-{눈}{강도0}1500-{눈}{강도0}1800-{눈}{강도0}2100-2315']

2017-01-30 00:00:00
['{눈}0015-0150']

2017-02-05 00:00:00
['{눈}0608-{눈}{강도0}0900-0940', '{비}

In [149]:
weather_info = dict()
is_before_day_not_end = False

for date in weather_dict.keys():
    in_dict = []
    w_arr = weather_dict[date]
    print(w_arr)
    weather = ""
    
    start_time = ""
    end_time = ""
    
    is_start = False
    is_end = False

    if is_before_day_not_end == True:
        start_time = "0000"
        print("is_before_day_not_end", start_time)
        is_before_day_not_end = False
        
    for w in w_arr:
        for idx,_ in enumerate(w):
            if (_ == "비") or (_ == "눈"):
                weather = _
                if len(start_time) == 0:
                    is_start = True
            if (is_start == True) and w[idx].isdigit():
#                 print(w[idx:])
                if (len(start_time) == 3) or (w[idx + 1].isdigit()):
                    start_time += _
                    if len(start_time) == 4:
                        is_start = False
                        
            if (_ == "-") and len(w) != (idx + 1) and (w[idx + 1].isdigit()):
                is_end = True
            if (is_end == True) and w[idx].isdigit():
#                 print(w[idx:])
                if (len(end_time) == 3) or (w[idx + 1].isdigit()):
                    end_time += _
                    if(len(end_time) == 4):
                        is_end = False
        print("{} {}\n".format(date, weather),
              "start time : {}\n".format(start_time),
             "end time : {}".format(end_time))
        if start_time != "":
            start_date = date.replace(hour=int(start_time[:2]), minute=int(start_time[2:]))
            end_date = "" if end_time == "" else \
                date.replace(hour=int(end_time[:2]), minute=int(end_time[2:]))
            
            in_dict.append({
                "weather": weather,
                "start time": start_date,
                "end time": end_date
            })
        if end_time != "":
            start_time = ""
            end_time = ""
        else:
            is_before_day_not_end = True
            print(" is_before_day_not_end : {}".format(w[-5:]))            
            
    weather_info[date] = in_dict

['{비}0410-0550', '{비}0725-0750']
2017-01-02 00:00:00 비
 start time : 0410
 end time : 0550
2017-01-02 00:00:00 비
 start time : 0725
 end time : 0750
['{비}1515-1545', '{비}1925-1950', '{비}2130-2250']
2017-01-08 00:00:00 비
 start time : 1515
 end time : 1545
2017-01-08 00:00:00 비
 start time : 1925
 end time : 1950
2017-01-08 00:00:00 비
 start time : 2130
 end time : 2250
['{비}0205-0220', '{비}0350-0445']
2017-01-09 00:00:00 비
 start time : 0205
 end time : 0220
2017-01-09 00:00:00 비
 start time : 0350
 end time : 0445
['{눈}0040-0220']
2017-01-12 00:00:00 눈
 start time : 0040
 end time : 0220
['{눈}0820-{눈}{강도0}0900-1145']
2017-01-13 00:00:00 눈
 start time : 0820
 end time : 1145
['{눈}1225-1315']
2017-01-18 00:00:00 눈
 start time : 1225
 end time : 1315
['{눈}0210-{눈}{강도0}0300-{눈}{강도1}0600-0855']
2017-01-20 00:00:00 눈
 start time : 0210
 end time : 0855
['{눈}0845-{눈}{강도0}0900-0915', '{눈}1125-1158', '{눈}1415-{눈}{강도0}1500-{시정(미만)}{1km}{눈}1715-{시정(이상)}{1km}{눈}1724-{눈}{강도0}1800-1845', '{눈}1945-1

 end time : 1650
['{비}0040-0115', '{비}0303-{비}{강도0}0600-0810']
2017-12-03 00:00:00 비
 start time : 0040
 end time : 0115
2017-12-03 00:00:00 비
 start time : 0303
 end time : 0810
['{비}0240-{비}{강도0}0300-0430']
2017-12-04 00:00:00 비
 start time : 0240
 end time : 0430
['{눈}2150-{눈}{강도0}2400-']
2017-12-05 00:00:00 눈
 start time : 2150
 end time : 
 is_before_day_not_end : 2400-
['{눈}-0120', '{눈}0150-0240', '{눈}1640-{시정(미만)}{1km}{눈}1745-{눈}{강도1}1800-{시정(이상)}{1km}{눈}1820-1855', '{눈}2050-{눈}{강도0}2100-{눈}2130-2140']
is_before_day_not_end 0000
2017-12-06 00:00:00 눈
 start time : 0000
 end time : 0120
2017-12-06 00:00:00 눈
 start time : 0150
 end time : 0240
2017-12-06 00:00:00 눈
 start time : 1640
 end time : 1855
2017-12-06 00:00:00 눈
 start time : 2050
 end time : 2140
['{눈}0435-{시정(미만)}{1km}{눈}0530-{눈}{강도1}0600-{시정(이상)}{1km}{눈}0650-{눈}{강도0}0900-{눈}0940-{비}1010-1120', '{비}1450-{비}{강도0}1500-1510', '{눈}2120-2145']
2017-12-10 00:00:00 비
 start time : 0435
 end time : 1120
2017-12-10 00:00:00 비


In [160]:
df_datas = pd.DataFrame(datas)[['날짜', '평균온도', '평균습도']]
df_datas.index = df_datas['날짜']
df_datas

,날짜,평균온도,평균습도
날짜,,,
2017-01-01,2017-01-01,2.7,75.9
2017-01-02,2017-01-02,5.0,77.8
2017-01-03,2017-01-03,2.0,61.8
2017-01-04,2017-01-04,3.9,55.0
2017-01-05,2017-01-05,3.8,52.3
...,...,...,...
2018-12-27,2018-12-27,-10.1,28.3
2018-12-28,2018-12-28,-10.5,33.0
2018-12-29,2018-12-29,-7.6,29.0


In [164]:
from pymongo import MongoClient as mc

mongo_uri = "mongodb://localhost:27017"
client = mc(mongo_uri)

weather_col = client.keti_pattern_recognition.weather_info

for idx in df_datas.index:
    in_weather = False
    try:
        in_weather = weather_info[idx]
    except:
        in_weather = False
    
    if in_weather == False:
        in_dict = dict({
            "date": idx,
            "avgTa": df_datas.loc[idx]['평균온도'],
            "avgRhm": df_datas.loc[idx]['평균습도']
        })
    else:
        in_dict = dict({
            "date": idx,
            "weather": in_weather,
            "avgTa": df_datas.loc[idx]['평균온도'],
            "avgRhm": df_datas.loc[idx]['평균습도']
        })
    weather_col.insert_one(in_dict)

client.close()